In [ ]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from sklearn import preprocessing


# функция нормализуем данные для работы
def normalization(data):
  mean = data.mean(axis=0)
  std = data.std(axis=0)
  data -= mean
  data /= std
  return data
  
# Возвращаем значения "у"
def get_truth(array):
  for col_name, col_info in array.items():
    if col_name == "y":
      data = []
      for line_number, line_info in col_info.items():
        data.append(line_info)
  return np.array(data, dtype=np.float64)

try:
  x_train = pd.read_csv("/content/drive/My Drive/ИАД/train.csv")
  x_test = pd.read_csv("/content/drive/My Drive/ИАД/test.csv")
  print("The data was imported")
except:
  print("Something went wrong")





The data was imported


In [ ]:

# Находим второй аргумент для метода .fit
truth = get_truth(x_train)

print(x_train)
print("\n\n")
print(x_test)



        ID       y  X0 X1  X2 X3 X4  ... X378 X379 X380  X382  X383  X384  X385
0        0  130.81   k  v  at  a  d  ...    0    0    0     0     0     0     0
1        6   88.53   k  t  av  e  d  ...    0    0    0     0     0     0     0
2        7   76.26  az  w   n  c  d  ...    0    0    0     1     0     0     0
3        9   80.62  az  t   n  f  d  ...    0    0    0     0     0     0     0
4       13   78.02  az  v   n  f  d  ...    0    0    0     0     0     0     0
...    ...     ...  .. ..  .. .. ..  ...  ...  ...  ...   ...   ...   ...   ...
4204  8405  107.39  ak  s  as  c  d  ...    0    0    0     0     0     0     0
4205  8406  108.77   j  o   t  d  d  ...    0    0    0     0     0     0     0
4206  8412  109.22  ak  v   r  a  d  ...    0    0    0     0     0     0     0
4207  8415   87.48  al  r   e  f  d  ...    0    0    0     0     0     0     0
4208  8417  110.85   z  r  ae  c  d  ...    0    0    0     0     0     0     0

[4209 rows x 378 columns]



        ID

In [ ]:
# Прогоняем данные через Label Encoder
df = pd.DataFrame(x_train)
x_train = df.apply(preprocessing.LabelEncoder().fit_transform)
df = pd.DataFrame(x_test)
x_test = df.apply(preprocessing.LabelEncoder().fit_transform)

print(x_train)
print("\n\n")
print(x_test)
print("\n\n")

        ID     y  X0  X1  X2  X3  X4  ...  X378  X379  X380  X382  X383  X384  X385
0        0  2466  32  23  17   0   3  ...     0     0     0     0     0     0     0
1        1   366  32  21  19   4   3  ...     0     0     0     0     0     0     0
2        2    69  20  24  34   2   3  ...     0     0     0     1     0     0     0
3        3   133  20  21  34   5   3  ...     0     0     0     0     0     0     0
4        4   106  20  23  34   5   3  ...     0     0     0     0     0     0     0
...    ...   ...  ..  ..  ..  ..  ..  ...   ...   ...   ...   ...   ...   ...   ...
4204  4204  1657   8  20  16   2   3  ...     0     0     0     0     0     0     0
4205  4205  1766  31  16  40   3   3  ...     0     0     0     0     0     0     0
4206  4206  1801   8  23  38   0   3  ...     0     0     0     0     0     0     0
4207  4207   280   9  19  25   5   3  ...     0     0     0     0     0     0     0
4208  4208  1921  46  19   3   2   3  ...     0     0     0     0     0     

In [ ]:
x_train_norm = normalization(x_train)
x_test_norm = normalization(x_test)
print(x_train_norm)
print(x_test_norm)

            ID         y        X0  ...     X383      X384      X385
0    -1.731434  1.822735  0.162993  ... -0.04081 -0.021801 -0.037778
1    -1.730611 -1.202695  0.162993  ... -0.04081 -0.021801 -0.037778
2    -1.729788 -1.630577 -0.710475  ... -0.04081 -0.021801 -0.037778
3    -1.728965 -1.538373 -0.710475  ... -0.04081 -0.021801 -0.037778
4    -1.728142 -1.577272 -0.710475  ... -0.04081 -0.021801 -0.037778
...        ...       ...       ...  ...      ...       ...       ...
4204  1.728142  0.657224 -1.583943  ... -0.04081 -0.021801 -0.037778
4205  1.728965  0.814259  0.090204  ... -0.04081 -0.021801 -0.037778
4206  1.729788  0.864682 -1.583943  ... -0.04081 -0.021801 -0.037778
4207  1.730611 -1.326593 -1.511154  ... -0.04081 -0.021801 -0.037778
4208  1.731434  1.037564  1.182039  ... -0.04081 -0.021801 -0.037778

[4209 rows x 378 columns]
            ID        X0        X1  ...      X383      X384     X385
0    -1.731434 -0.625137  1.395595  ... -0.021801 -0.026704 -0.04081
1    -1

In [ ]:
x_train_norm.drop('y', axis='columns', inplace=True)
print(x_train_norm)

            ID        X0        X1  ...     X383      X384      X385
0    -1.731434  0.162993  1.393322  ... -0.04081 -0.021801 -0.037778
1    -1.730611  0.162993  1.158883  ... -0.04081 -0.021801 -0.037778
2    -1.729788 -0.710475  1.510542  ... -0.04081 -0.021801 -0.037778
3    -1.728965 -0.710475  1.158883  ... -0.04081 -0.021801 -0.037778
4    -1.728142 -0.710475  1.393322  ... -0.04081 -0.021801 -0.037778
...        ...       ...       ...  ...      ...       ...       ...
4204  1.728142 -1.583943  1.041664  ... -0.04081 -0.021801 -0.037778
4205  1.728965  0.090204  0.572786  ... -0.04081 -0.021801 -0.037778
4206  1.729788 -1.583943  1.393322  ... -0.04081 -0.021801 -0.037778
4207  1.730611 -1.511154  0.924444  ... -0.04081 -0.021801 -0.037778
4208  1.731434  1.182039  0.924444  ... -0.04081 -0.021801 -0.037778

[4209 rows x 377 columns]


In [ ]:
# создаем нейронную сеть
model = Sequential()
model.add(Dense(512, activation="relu", input_shape=(x_train_norm.shape[1],)))
model.add(Dense(256, activation="relu"))
model.add(Dense(128, activation="relu"))
model.add(Dense(64, activation="relu"))
model.add(Dense(32, activation="relu"))
model.add(Dense(16, activation="relu"))
model.add(Dense(8, activation="relu"))
model.add(Dense(4, activation="relu"))
model.add(Dense(2, activation="relu"))
model.add(Dense(1))

try:
  model.compile(optimizer="adam", loss="mse", metrics=["mae"])
  print("Compiled successfully")
except:
  print("Couldn`t compile the NN")

Compiled successfully


In [ ]:
# начинаем обучение нейронной сети
try:
  model.fit(x_train_norm, truth, epochs=400, batch_size=100, verbose=1) 
  print("Success")
except:
  print("Couldn`t train the NN")

Epoch 1/400
43/43 [==============================] - 0s 4ms/step - loss: 10290.9199 - mae: 100.6488
Epoch 2/400
43/43 [==============================] - 0s 4ms/step - loss: 10282.2646 - mae: 100.6059
Epoch 3/400
43/43 [==============================] - 0s 4ms/step - loss: 10273.6289 - mae: 100.5629
Epoch 4/400
43/43 [==============================] - 0s 4ms/step - loss: 10264.9893 - mae: 100.5200
Epoch 5/400
43/43 [==============================] - 0s 4ms/step - loss: 10256.3496 - mae: 100.4770
Epoch 6/400
43/43 [==============================] - 0s 4ms/step - loss: 10247.7246 - mae: 100.4341
Epoch 7/400
43/43 [==============================] - 0s 4ms/step - loss: 10239.0977 - mae: 100.3911
Epoch 8/400
43/43 [==============================] - 0s 4ms/step - loss: 10230.4785 - mae: 100.3481
Epoch 9/400
43/43 [==============================] - 0s 4ms/step - loss: 10221.8643 - mae: 100.3052
Epoch 10/400
43/43 [==============================] - 0s 4ms/step - loss: 10213.2656 - mae: 100.2623

In [ ]:
mse, mae = model.evaluate(x_train_norm, x_test_norm, verbose=1) 
print("Squared error is", mse)
print("Absolute error is", mae)

132/132 [==============================] - 0s 3ms/step - loss: nan - mae: nan
Squared error is nan
Absolute error is nan


In [ ]:
predictions = model.predict(x_test_norm, verbose=1)
max_arg = np.argmax(predictions[0])
print("Predicted", max_arg)

132/132 [==============================] - 0s 2ms/step


 PREDICTED ARGUMENT: 0
